In [240]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import itertools
import time

In [266]:
start_indexes = ["1", "26", "51", "76"]
book_codes = []
book_titles = []
book_content = []
book_authors = []
book_downloads = []

In [267]:
# Create proxies to prevent hsot from blocking requests
proxies = {'http': 'http://176.110.121.90:21776'} 

In [268]:
# Get book data

# Pages only include 25 books so need to loop 4 times to get 100 books
for start_index in start_indexes:
    # Read website html
    url = "https://www.gutenberg.org/ebooks/search/?sort_order=downloads&start_index=" + start_index
    response = requests.get(url,proxies=proxies)
    books_soup = BeautifulSoup(response.text, 'html.parser')
   
    book_link_soup = books_soup.find_all('li', attrs={'class': 'booklink'})
    
    for book_link in book_link_soup:
        #Get book titles
        title= book_link.find('span', attrs={'class': 'title'}).getText()
        book_titles.append(" ".join(title.split()))
        
        #Get book author
        author = book_link.find('span', attrs={'class': 'subtitle'})
        if author is not None:
            book_authors.append(" ".join(author.getText().split()))
        else:
            book_authors.append("")
        
        #Get book downloads
        downloads = book_link.find('span', attrs={'class': 'extra'}).getText()
        book_downloads.append(int(downloads.split()[0]))
    
        # Get book codes
        book_codes_soup = book_link.find('a', attrs={'class': 'link'})
        
        # Remove /ebooks/ and non numbers
        book_code = book_codes_soup.attrs['href'][8:]
        book_codes.append(book_code)

        # Get first 10 paragraphs from book
        chapter_url = "https://www.gutenberg.org/files/" + book_code + "/" + book_code + "-h/" + book_code + "-h.htm"
        chapter_response = requests.get(chapter_url,proxies=proxies)
        chapter_soup = BeautifulSoup(chapter_response.text, 'html.parser')

        paragraphs = chapter_soup.find_all('p', text=True, limit=10)
        text = ""
        
        for i, paragraph in enumerate(paragraphs):
            paragraphs[i] = paragraph.getText()
            paragraphs[i] = " ".join(paragraphs[i].split())
            text = text + " " + paragraphs[i]

        book_content.append(text)

In [269]:
# Construct a dataframe
df = pd.DataFrame({'book_code': book_codes, 'book_title': book_titles, 'book_author': book_authors, 'book_downloads': book_downloads, 'book_content': book_content})
df.head()

,book_code,book_title,book_author,book_downloads,book_content
0,1342,Pride and Prejudice,Jane Austen,54222,"It is a truth universally acknowledged, that ..."
1,84,"Frankenstein; Or, The Modern Prometheus",Mary Wollstonecraft Shelley,43135,"St. Petersburgh, Dec. 11th, 17—. You will rej..."
2,11,Alice's Adventures in Wonderland,Lewis Carroll,27199,Alice was beginning to get very tired of sitt...
3,16328,Beowulf: An Anglo-Saxon Epic Poem,,25709,CONTENTS. The Heyne-Socin text and glossary h...
4,25344,The Scarlet Letter,Nathaniel Hawthorne,25582,BY NATHANIEL HAWTHORNE. Illustrated. The auth...


In [270]:
# Put into .csv file
df.to_csv('top100books.csv', index=False, encoding="utf-8-sig")